# Data Quality and consistency checks

# Contents
# 1. Investigate and understand new df
# 2. Make consistency changes
# 3. Check for mixed types, duplicates, missing values
# 4. Merge dfs

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

# def file path
path = r'/Users/alison/Documents/Data Analysis - Tasks/Instacart  Basket Analysis' 
#get df ords_prods_merged
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'newest_ords_prods_merged.pkl'))

In [2]:
#get customer df
df_custs = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col=False)

# 1. Investigate and understand new df

In [5]:
df_custs.shape

(206209, 10)

In [7]:
df_custs.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [9]:
df_custs.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

# 2. Make consistency changes

In [11]:
# change redundant dtypes, user_id same as ords_prods_merge, change column name - Surnam to Last_name
df_custs.rename(columns = {'Surnam' : 'Last_name'}, inplace = True)

In [13]:
df_custs['user_id'] = df_custs['user_id'].astype('str')

In [15]:
df_custs['user_id'].dtype

dtype('O')

In [17]:
# check changes 
df_custs

,user_id,First Name,Last_name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
...,...,...,...,...,...,...,...,...,...,...
206204,168073,Lisa,Case,Female,North Carolina,44,4/1/2020,1,married,148828
206205,49635,Jeremy,Robbins,Male,Hawaii,62,4/1/2020,3,married,168639
206206,135902,Doris,Richmond,Female,Missouri,66,4/1/2020,2,married,53374
206207,81095,Rose,Rollins,Female,California,27,4/1/2020,1,married,99799


# 2. Make consistency changes

In [19]:
# check latest merged df
ords_prods_merge.dtypes

product_id                   int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
New_customer_flag           object
add_to_cart_order            int64
reordered                    int64
_merge                    category
price_range_loc             object
Busiest_days                object
busiest period of day       object
max_order                    int64
loyalty_flag                object
av_product_price           float64
spending_flag               object
Frequent customer          float64
frequency_flag              object
dtype: object

In [21]:
ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('str')

In [22]:
ords_prods_merge['order_id'] = ords_prods_merge['order_id'].astype('str')

In [24]:
# check changes
ords_prods_merge.dtypes

product_id                   int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
order_id                    object
user_id                     object
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
New_customer_flag           object
add_to_cart_order            int64
reordered                    int64
_merge                    category
price_range_loc             object
Busiest_days                object
busiest period of day       object
max_order                    int64
loyalty_flag                object
av_product_price           float64
spending_flag               object
Frequent customer          float64
frequency_flag              object
dtype: object

# 3. Check for mixed types, duplicates, missing values

In [27]:
#check for mixed types
for col in df_custs.columns.tolist():
  weird = (df_custs[[col]].map(type) != df_custs[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_custs[weird]) > 0:
    print (col)

First Name


In [29]:
# First name has mixed types change to str
df_custs['First Name'] = df_custs['First Name'].astype('str')

In [31]:
#check for missing values
df_custs.isnull().sum()

user_id         0
First Name      0
Last_name       0
Gender          0
STATE           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [33]:
# no missing values. check for duplicates
df_custs_clean = df_custs[df_custs['Last_name'].isnull() == False]
df_dups = df_custs_clean[df_custs_clean.duplicated()]

In [35]:
df_dups

,user_id,First Name,Last_name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income


# 4. Merge dfs

In [37]:
# rename merge column as stopping join on tables
ords_prods_merge.rename(columns = {'_merge' : 'old_merge'}, inplace = True)

In [39]:
ords_prods_merge

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,old_merge,price_range_loc,Busiest_days,busiest period of day,max_order,loyalty_flag,av_product_price,spending_flag,Frequent customer,frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,both,Mid-range product,Regularly busy,most_orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,both,Mid-range product,Regularly busy,average orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,both,Mid-range product,Busiest days,average orders,5,New customer,7.930208,Low spender,8.0,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,both,Mid-range product,Slowest days,most_orders,3,New customer,4.972414,Low spender,9.0,Frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,both,Mid-range product,Slowest days,average orders,3,New customer,4.972414,Low spender,9.0,Frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,49688,Fresh Foaming Cleanser,73,11,13.5,1788356,200215,2,0,9,...,both,Mid-range product,Busiest days,most_orders,6,New customer,8.220313,Low spender,7.0,Frequent customer
32404855,49688,Fresh Foaming Cleanser,73,11,13.5,3401313,200377,1,4,11,...,both,Mid-range product,Slowest days,most_orders,4,New customer,7.364516,Low spender,30.0,Non-frequent customer
32404856,49688,Fresh Foaming Cleanser,73,11,13.5,809510,200873,5,3,8,...,both,Mid-range product,Slowest days,average orders,20,Regular customer,8.500344,Low spender,6.0,Frequent customer
32404857,49688,Fresh Foaming Cleanser,73,11,13.5,2359893,200873,9,3,15,...,both,Mid-range product,Slowest days,most_orders,20,Regular customer,8.500344,Low spender,6.0,Frequent customer


In [41]:
# merge tables on common column ensuring dtype is same
df_merged_all = df_custs.merge(ords_prods_merge, on = 'user_id', indicator = True)

In [43]:
df_merged_all['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [45]:
# check dimensions
df_merged_all.shape

(32404859, 34)

In [47]:
#export df_merged_all as pkl
df_merged_all.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'merged_all.pkl'))